In [12]:
import tensorflow as tf
from preprocess import play_midi, parse_file,get_data_from_dir

Contains special words for the dictionary

In [29]:
special_words = ["<PAD>", "<GO>", "<END>"]

## Class For Dictionary
The class contains the data and the helper functions for the dictionary to train seq2seq model.
Most important functions are to mapping a list of words(a sentence) to a list of corresponding integer(indeces of words). 

In [34]:
class Seq2Seq_Dictionary:
    def __init__(self, sentences ):
        self.word2index_map = dict()
        self.index2word_map = list()
        self.init_register(sentences)
        
    # Initiates word2index_map and index2word_map
    # Also extracts the max number of words in the sentences and saves it 
    def init_register(self,sentences):
        global special_words
        current_index = 0
        ## save the maximum length among the sentences. 
        self.max_length = max([len(sentence) for sentence in sentences])
        ### map special words, initially the mappings are empty.
        for word in special_words:
            self.word2index_map[word] = current_index
            self.index2word_map.append(word)
            current_index+=1
        
        ## iterate input sentences
        for sentence in sentences:
            
            #iterate the words in a sentence    
            for word in sentence:
                # if the word is not in the map, add it to both maps.
                # to count the index current_index is used.
                if word not in self.word2index_map:
                    self.word2index_map[word] = current_index
                    self.index2word_map.append(word)
                    current_index+=1
    
    ## Returns the index of the word in the dictionary. It is assumed that the word
    ## will be always in dictionary.
    def get_index(self, word):
        return self.word2index_map[word]
    
    ## Maps a sentence, which is a list of words, to the corresponding list of integers.
    ## Each word is looked up from the map of the dictionary, and as in get_index method,
    ## it is assumed that the word will always be found in the dictionary
    def map_sentence(self, sentence):
        mapping = list()
        for word in sentence:
            mapping.append(self.get_index(word))
        return mapping
    
    ## Returns the word by its index in dictionary.
    #def get_word(self, index):
    #    return self.index2word_map[word]
    
    ### Reverse operation of map_sentence. Gets the sentence, list of integers, as input and
    ### maps each entry from index to word
    #def decode_indeces(self,sentence_with_index):
    #    sentence_list = list()
        

#### Create the dictionary

In [35]:
data_dict = Seq2Seq_Dictionary([["asdad","asdad","asdada"]])

## Model Creation
Tensorflow initialization

In [45]:
#sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()


### General Model Variables 

In [33]:
PAD = 0
START = 1
END = 2

embedding_size= 100
encoder_hidden_units = 128
decoder_hidden_units = 128
keep_prob=0.5 # Dropout parameter
batch_size = 64
max_seq_length = data_dict.max_length
vocab_size = len(data_dict.index2word_map)

#### Network IO

In [46]:
_inputs = tf.placeholder(shape=(batch_size, max_seq_length), dtype=tf.int32, name='Inputs')
_targets = tf.placeholder(shape=(batch_size, None), dtype=tf.int32, name='Targets')

Encoder part is created here. In the architecture, a bidirectional LSTM cell is used after the embedding.

In [47]:
with tf.variable_scope("encoder") as encoder_sc:
    ## embeddings
    enc_embed_var = tf.Variable(
        tf.random_uniform([vocab_size,
                           embedding_size],
                          -1.0, 1.0), name='embedding')
    
    enc_embed = tf.nn.embedding_lookup(enc_embed_var, _inputs)
    
    # Forward direction cell
    enc_lstm_fw = tf.nn.rnn_cell.LSTMCell(encoder_hidden_units, forget_bias=1.0)
    # Backward direction cell
    enc_lstm_bw = tf.nn.rnn_cell.LSTMCell(encoder_hidden_units, forget_bias=1.0)
    
    enc_dropout_fw = tf.contrib.rnn.DropoutWrapper(enc_lstm_fw, input_keep_prob=keep_prob,
                                                   output_keep_prob=keep_prob)

    enc_dropout_bw = tf.contrib.rnn.DropoutWrapper(enc_lstm_bw, input_keep_prob=keep_prob,
                                                   output_keep_prob=keep_prob)

    
    ## here the state variable contains only the last state information of the cells
    enc_rnn_outputs,enc_rnn_state=tf.nn.bidirectional_dynamic_rnn(enc_dropout_fw,
                                                                  enc_dropout_bw, 
                                                                  enc_embed, dtype=tf.float32)
    enc_rnn_las_fw,enc_rnn_outputs_bw  = enc_rnn_outputs
    enc_rnn_fw_state,enc_rnn_bw_state  = enc_rnn_state
    
    